In [60]:
import torch
from torch import nn
from torch.nn import functional as F


def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out')
        nn.init.constant_(m.bias, 0.0)
    elif classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
    elif classname.find('BatchNorm') != -1:
        if m.affine:
            nn.init.normal_(m.weight, 1.0, 0.02)
            nn.init.constant_(m.bias, 0.0)


class Feature_Fusion_Module(nn.Module):
    # 自定义特征融合模块
    def __init__(self, ** kwargs):
        super(Feature_Fusion_Module, self).__init__()
        # Classifier for each stripe
        self.fc1 = nn.Linear(512, 6)
        self.fc1.apply(weights_init_kaiming)


    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x


ffm = Feature_Fusion_Module()
print(ffm)
rd = torch.randn(batchsize, 512)
print(rd.shape)
res = ffm(rd)
print(res)
print(res.shape)
print('complete check.')

Feature_Fusion_Module(
  (fc1): Linear(in_features=512, out_features=6, bias=True)
)
torch.Size([8, 512])
tensor([[3.4649e-02, 9.6369e-01, 1.0000e+00, 1.0000e+00, 9.9979e-01, 9.2831e-03],
        [1.5973e-13, 9.9944e-01, 1.1512e-07, 1.0000e+00, 9.9882e-01, 8.8866e-13],
        [9.9998e-01, 5.5238e-03, 1.9316e-11, 1.1402e-02, 3.1090e-13, 1.5034e-04],
        [1.0000e+00, 1.8289e-03, 6.5744e-07, 4.3585e-02, 3.8857e-12, 1.0000e+00],
        [3.7716e-06, 1.0000e+00, 9.9625e-01, 9.9985e-01, 2.7275e-11, 1.0000e+00],
        [2.1599e-01, 2.6010e-01, 9.9918e-01, 2.8339e-02, 4.8816e-02, 1.0000e+00],
        [1.0000e+00, 9.8869e-01, 6.9431e-06, 3.8659e-04, 8.7119e-01, 2.3240e-04],
        [9.9999e-01, 9.9997e-01, 9.9160e-01, 4.6477e-02, 2.2280e-14, 8.1987e-01]],
       grad_fn=<SigmoidBackward>)
torch.Size([8, 6])
complete check.


In [66]:
res[:][0]

tensor([0.0346, 0.9637, 1.0000, 1.0000, 0.9998, 0.0093],
       grad_fn=<SelectBackward>)

In [10]:
import torch
from torch import nn
from torch.nn import functional as F

In [42]:
batchsize=8

In [24]:
features_H = []  # contains 6 ([N, 256, 1])
for i in range(6):
    stripe_features_H = torch.randn(8,256,1)
    features_H.append(stripe_features_H)

In [15]:
mult_factor = torch.randn(8,6)
mult_factor

tensor([[ 2.3334,  0.9977,  0.8775,  0.4639,  0.0664, -1.1411],
        [ 1.2668, -0.6134, -1.5213, -1.0079,  0.1921, -0.5048],
        [ 0.6378,  1.6792,  1.6342, -1.1045, -0.8049, -0.3469],
        [ 0.9840,  1.0953,  1.1998,  0.5224,  0.6483, -2.3979],
        [ 0.2647,  1.1764, -0.1680,  0.2378,  0.4519,  0.0712],
        [ 2.0797,  0.2940, -0.7968,  0.0872, -0.9600, -0.4466],
        [ 0.7881,  0.8804, -1.4072, -0.8505,  1.8725,  0.2865],
        [-0.5789,  0.2964,  1.5520, -0.4665,  1.1704,  1.0765]])

In [31]:
mult_factor[0].shape

torch.Size([6])

In [40]:
mult_factor[:][0].shape

torch.Size([6])

In [34]:
features_H[0].shape
features_H[0].view(8,-1).shape
(features_H[0]*mult_factor[0][0]).shape

torch.Size([8, 256, 1])

In [84]:
for i in range(6):
    print((features_H[i].view(batchsize,-1)*mult_factor[:][i]).shape)
    print(features_H[i].view(batchsize,-1)*mult_factor[:,i])

RuntimeError: The size of tensor a (256) must match the size of tensor b (6) at non-singleton dimension 1

In [58]:
features_H[i].view(batchsize,-1).shape
mult_factor[:][0].shape
mult_factor[:][0]

tensor([ 2.3334,  0.9977,  0.8775,  0.4639,  0.0664, -1.1411])

In [73]:
features_H[0].shape

torch.Size([8, 256, 1])

In [78]:
mult_factor[:,1].shape

torch.Size([8])

In [80]:
mult_factor

tensor([[ 2.3334,  0.9977,  0.8775,  0.4639,  0.0664, -1.1411],
        [ 1.2668, -0.6134, -1.5213, -1.0079,  0.1921, -0.5048],
        [ 0.6378,  1.6792,  1.6342, -1.1045, -0.8049, -0.3469],
        [ 0.9840,  1.0953,  1.1998,  0.5224,  0.6483, -2.3979],
        [ 0.2647,  1.1764, -0.1680,  0.2378,  0.4519,  0.0712],
        [ 2.0797,  0.2940, -0.7968,  0.0872, -0.9600, -0.4466],
        [ 0.7881,  0.8804, -1.4072, -0.8505,  1.8725,  0.2865],
        [-0.5789,  0.2964,  1.5520, -0.4665,  1.1704,  1.0765]])

In [83]:
mult_factor[:,0],mult_factor[:,0].shape,features_H[i].view(batchsize,-1).shape

(tensor([ 2.3334,  1.2668,  0.6378,  0.9840,  0.2647,  2.0797,  0.7881, -0.5789]),
 torch.Size([8]))

In [88]:
features_H[i].view(batchsize,-1)*mult_factor[:,0]

RuntimeError: The size of tensor a (256) must match the size of tensor b (8) at non-singleton dimension 1

In [89]:
mult_factor[:,0].expand_as(features_H[i].view(batchsize,-1))

RuntimeError: The expanded size of the tensor (256) must match the existing size (8) at non-singleton dimension 1.  Target sizes: [8, 256].  Tensor sizes: [8]

In [92]:
features_H[i].view(batchsize,-1)*mult_factor[:,0].view(batchsize,-1).expand(-1,256)

tensor([[-0.5531, -0.9327,  0.2130,  ...,  0.8494,  2.3303,  2.2622],
        [-0.2042, -0.1271,  0.4650,  ..., -0.4667, -2.0964, -1.0976],
        [ 0.5501,  0.3041, -0.4537,  ...,  0.1158,  0.3765, -0.1287],
        ...,
        [ 0.4249, -2.6174, -1.7900,  ...,  1.4845,  0.6306, -1.7327],
        [ 0.3420, -0.3949,  0.6263,  ..., -0.5655, -0.8899,  0.3292],
        [-0.3473,  0.2408, -0.3961,  ...,  0.3422, -0.5082, -0.0835]])

In [94]:
mult_factor[:,0].view(batchsize,-1).expand(-1,256),features_H[i].view(batchsize,-1)

(tensor([[ 2.3334,  2.3334,  2.3334,  ...,  2.3334,  2.3334,  2.3334],
         [ 1.2668,  1.2668,  1.2668,  ...,  1.2668,  1.2668,  1.2668],
         [ 0.6378,  0.6378,  0.6378,  ...,  0.6378,  0.6378,  0.6378],
         ...,
         [ 2.0797,  2.0797,  2.0797,  ...,  2.0797,  2.0797,  2.0797],
         [ 0.7881,  0.7881,  0.7881,  ...,  0.7881,  0.7881,  0.7881],
         [-0.5789, -0.5789, -0.5789,  ..., -0.5789, -0.5789, -0.5789]]),
 tensor([[-0.2370, -0.3997,  0.0913,  ...,  0.3640,  0.9987,  0.9695],
         [-0.1612, -0.1003,  0.3671,  ..., -0.3684, -1.6549, -0.8664],
         [ 0.8624,  0.4768, -0.7113,  ...,  0.1816,  0.5903, -0.2018],
         ...,
         [ 0.2043, -1.2585, -0.8607,  ...,  0.7138,  0.3032, -0.8332],
         [ 0.4340, -0.5011,  0.7948,  ..., -0.7176, -1.1292,  0.4178],
         [ 0.5998, -0.4159,  0.6842,  ..., -0.5911,  0.8778,  0.1442]]))

In [95]:
2.3334*-0.2370

-0.5530158000000001

In [98]:
features_H[i].view(batchsize,-1)*mult_factor[:,0].view(batchsize,-1).expand(features_H[i].view(batchsize,-1).shape)

tensor([[-0.5531, -0.9327,  0.2130,  ...,  0.8494,  2.3303,  2.2622],
        [-0.2042, -0.1271,  0.4650,  ..., -0.4667, -2.0964, -1.0976],
        [ 0.5501,  0.3041, -0.4537,  ...,  0.1158,  0.3765, -0.1287],
        ...,
        [ 0.4249, -2.6174, -1.7900,  ...,  1.4845,  0.6306, -1.7327],
        [ 0.3420, -0.3949,  0.6263,  ..., -0.5655, -0.8899,  0.3292],
        [-0.3473,  0.2408, -0.3961,  ...,  0.3422, -0.5082, -0.0835]])

In [100]:
t = torch.randn(3,4)
t

tensor([[ 0.0453,  0.3860,  0.0805,  0.8701],
        [-1.2264, -0.3673, -0.0276, -1.6460],
        [ 0.2817, -0.4221,  0.4141, -0.3153]])

In [104]:
t[0][0],t[:][0],t[:,0]

(tensor(0.0453),
 tensor([0.0453, 0.3860, 0.0805, 0.8701]),
 tensor([ 0.0453, -1.2264,  0.2817]))

In [106]:
t[0][:],t[0,:]

(tensor([0.0453, 0.3860, 0.0805, 0.8701]),
 tensor([0.0453, 0.3860, 0.0805, 0.8701]))

In [4]:
(64.7-56.2 )/56.2 

0.15124555160142347